In [235]:
import numpy as np
from as_pysrc.svp_cmds import alignSnds, calc_f0_swipe
from as_pysrc.svp_cmds.transpose_f0 import transpose_f0
import as_pysrc.fileio.sdif.FSdifLoadFile as FSdifLoadFile
from as_pysrc.fileio.sdif.get_f0_info import get_f0_info
from scipy.io import wavfile, loadmat
import os
import glob
from shutil import copyfile

import sys
print(sys.path)

['', '/Users/robinson/Dropbox/anasynth/_code', '/Users/robinson/Dropbox/anasynth/_code/as_pysrc', '/Users/robinson/Dropbox/anasynth/_code/tf_seq2seq', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python36.zip', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python3.6', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python3.6/lib-dynload', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages', '/Users/robinson/Dropbox/anasynth/_code/seq2seq', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/as_pysrc_utils_poly-0.2-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/as_pysrc_utils_levinson-0.1-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/utils_bspline-1.0.0-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/opt_congrad_sc_c-1.2.2

In [236]:
# **** SETTINGS ********************************************

# set harmonicity_threshold
harmonicity_threshold = 0.3

# set interpolation to True/False
# interp = False
interp = True

# directory where phonemes are kept, as per phon_input_directory_path
dirr = '20180829_131313_col8_subcond'

# e.g. '.JOY_1' - must include .
emotion_code = '.COL_8'

# if true, calc_f0_swipe is run on original WAVs (slow)
regen_orig_f0 = False

<h3>Remember: Make syllable F0 CSV files for use during WAV conversion using tf_seq2seq_data_processing_test_input.ipynb

In [237]:
# def alignSnds(sndfile_to_be_aligned, outfile, temporal_positions,
#               winsize,  svpcmd=svp_executable, shape=False,
#               transients=False, oversamp=8, vudem=3, transdet_minren=None, verbose=False) :
#     '''
#     use supervp to align sndfile_to_be_aligned to sndfile by means of stretching
#     the first to align to the second.
#     '''
    
'''
# I need a way to link the transformed syllable f0 back to it's source file and position in that file, 
so I can assemble the sequence of transformed f0 contours for a particular phrase

# the problem with test data is that the syllables are a subset of all syllables across all files in the dataset, 
# so not all syllables of a particular phrase will be present.. This makes it difficult to infer only on test data 
in order to test the conversion.

# I could 'cheat', and just infer on the whole dataset (including data i trained on), to see what the conversion is 
like - DONE
# Or I could separate off some complete phrases to test on later (random shuffle), and train on the rest. - TODO 
# would be good to do both, to see the difference it makes! 


# for each wavfile...

# get list of syllables, and make list of true/false to indicate voiced/unvoiced

# make list of before-times (all the start and end times of all voiced and unvoiced parts) - this is just what's in 
the mat file (syll_time) - you prob don't want the final value

# make list of end-times (all the start and end times of all voiced and unvoiced parts)
    # make list of durations of each voiced syllable from new f0 contour, and duration of unvoiced syllable from 
    original file (handle missing files too..):
    # for each syllable in a file:
      # if syll is voiced
          # if file is present:
            # add NEW f0 contour length to dur-list
          # else add original 'before-time' syll length
      # else if unvoiced
          # add before-times unvoiced syll length to dur-list

# make a new list for the final end-times
# for all elements in dur-list
  # add end time to end-times-list as sum of all elements currently in the list, plus next duration in the list - 
  cumsum
          


'''

"\n# I need a way to link the transformed syllable f0 back to it's source file and position in that file, \nso I can assemble the sequence of transformed f0 contours for a particular phrase\n\n# the problem with test data is that the syllables are a subset of all syllables across all files in the dataset, \n# so not all syllables of a particular phrase will be present.. This makes it difficult to infer only on test data \nin order to test the conversion.\n\n# I could 'cheat', and just infer on the whole dataset (including data i trained on), to see what the conversion is \nlike - DONE\n# Or I could separate off some complete phrases to test on later (random shuffle), and train on the rest. - TODO \n# would be good to do both, to see the difference it makes! \n\n\n# for each wavfile...\n\n# get list of syllables, and make list of true/false to indicate voiced/unvoiced\n\n# make list of before-times (all the start and end times of all voiced and unvoiced parts) - this is just what's in \

<h3>Make SDIFs containing F0 for original WAV files

In [238]:
# input directory is former output_directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO'
input_directory_path = wav_input_directory_path
input_directory = os.fsencode(input_directory_path)

# define new output directory and files
output_directory_path = os.path.join(wav_input_directory_path, 'original_f0extract')
if not os.path.exists(output_directory_path):
    os.mkdir(output_directory_path)

if regen_orig_f0:
    # for each wav file in directory
    for file in os.listdir(input_directory):
        # get filename
        filename = os.fsdecode(file)
        # get filepath and decode from bytes to string
        # filepath = os.path.abspath(file).decode("utf-8") 
        filepath = os.path.join(input_directory_path, filename)
        print(filepath)
        
        if filename.endswith('.wav'):
            # build outfile sdif filename
            sdif_file_out, ext = os.path.splitext(filename)
            sdif_file_out = sdif_file_out + '.sdif'
            sdif_file_out_path = os.path.join(output_directory_path, sdif_file_out)
            
            # extract f0 and write to sdif
            calc_f0_swipe(filepath, sdif_file_out_path)
        
print('done')

done


<h3>Generate before and after syllable timecode csv files, and minimum F0 file

In [239]:
# path to directory for test_log.txt
csv_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006/f0_raw_phoneme'
# Olivia2006.e01.p01.i00.s01_s e1.csv
csv_input_file_extension = '.csv'

# path to predicted F0 CSV input files directory
phon_input_directory_path = '/Users/robinson/Downloads/data/pred/' + dirr + '/phonemes'
# Olivia2006.e01.p01.i00.s01_s e1.csv
phon_input_file_extension = '.csv'

# path to MAT input files directory
mat_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006'
# Olivia2006.e01.p01.i00.mat
mat_input_file_extension = '.mat'

# path to WAV input files directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO'
# Olivia2006.e01.p01.i00.1.wav
wav_input_file_extension = '.wav'

# path to SDIF f0 input files directory
sdif_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/original_f0extract'
# Olivia2006.e01.p01.i00.mat
sdif_input_file_extension = '.sdif'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# set window step time 
step_s = 0.005  # 5ms, which I assume is what was used to sample the file

# define output directory and files
output_directory = os.path.join(wav_input_directory_path, 'timestretch')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# define phrase range
phrase_from = 1
phrase_to = 10
phrase_list = [1, 3, 5, 10]
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# open test_log.txt and split into list of lines
f_test_log = open(os.path.join(csv_input_directory_path, 'out', 'test_log.txt'), 'r')
test_log = f_test_log.read().split('\n')
# print(test_log)

# output filenames
filename_wav_files = 'wav_files.txt'
filename_phon_time = 'phon_time.txt'
filename_new_phon_time = 'phon_time_new.txt'
filename_phon_vals_min = 'phon_vals_min.txt'
# open output files in subdirectory of input files directory (must create manually)
f_wav_files = open(os.path.join(output_directory, filename_wav_files), 'w')
f_phon_time = open(os.path.join(output_directory, filename_phon_time), 'w')
f_new_phon_time = open(os.path.join(output_directory, filename_new_phon_time), 'w')
f_phon_vals_min = open(os.path.join(output_directory, filename_phon_vals_min), 'w')

# for each wavfile that we want to treat..
# for each phrase
# for p in range(phrase_from, phrase_to + 1):
for p in phrase_list: 
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            # to handle the number at the end, get all files beginning with name (there should only be one)
            glob_filepath = os.path.join(wav_input_directory_path, filename_base + '.*')
            wav_filepath_list = glob.glob(glob_filepath)
            wav_filepath = wav_filepath_list[0]
            # print(wav_filepath)
            _, wav_filename = os.path.split(wav_filepath)
            # print('*********************')
            # print('******** WAV ********')
            # print('wav_filename) ', wav_filename)

            # load corresponding mat file
            mat_filename = ''.join([filename_base, mat_input_file_extension])
            mat_filepath = os.path.join(mat_input_directory_path, mat_filename)
            # print(mat_filepath)
            mat_dict = loadmat(mat_filepath)
            
            syll_label = mat_dict['syll_label']
            syll_label = syll_label.reshape((syll_label.shape[1],))        
            # print(syll_label.shape)
            # print(syll_label)
            # syll_time.shape (2, 11)
            # I want syll_time.shape (11, 2) BUT with the start and end times in different 'columns' - just transpose!
            syll_time = mat_dict['syll_time']
            # print('syll_time.shape', syll_time.shape)
            syll_time = syll_time.T            
            
            # get list of phonemes
            phon_label = mat_dict['phone_label']
            phon_label = phon_label.reshape((phon_label.shape[1],))
            
            # make list of before-times (all the start and end times of all vowels and nonvowel parts) - this is just what's in 
            # the mat file (phon_time) - you prob don't want the final value. Also, phoneme times are not 
            # consecutive - one phoneme ends, then there's a gap, then the next begins. So you have to handle the 
            # start and end times of both
            # get list of phoneme start/end times
            phon_time = mat_dict['phone_time']
            # print(phon_time.shape)
            # print(phon_time)
            # phon_time = phon_time.reshape((phon_time.shape[1], phon_time.shape[0]))  # wrong!
            phon_time = phon_time.T # much better
            # print(phon_time.shape)
            # print(phon_time)
            # print(phon_time.shape) # (11, 2)
            # print('phon_time = ', phon_time)
            # print(type(phon_time)) # <class 'numpy.ndarray'>
            
            # make list of true/false to indicate vowels(true)/nonvowel(false)
            vowel_phonemes = ['e~', '9~', 'a~', 'o~', 'i', 'e', 'E', 'a', 'A', 'O', 'o', 'u', 'y', '2', '9', '@']
            # [a if C else b for i in items]
            vowels = [True if phon[0] in vowel_phonemes else False for i, phon in enumerate(phon_label)]
            # print('vowels ', vowels)
            
            # make new_phon_time list as flattened version of original start and end times, to modify later
            # new_phon_time = list(phon_time.flatten(order='C'))
            # print('phon_time = ', new_phon_time)
            new_phon_time = [0.0] + [x for x in phon_time[:, 1]]  # just get second column, plus add a 0.0 at the start
            # print(new_phon_time)
            
            # create empty list to store all phon_vals, to later calculate the minimum f0 in the vowels regions
            # phon_vals_all = []
            # for each phoneme in the wav file
            for all_phon_counter, phon in enumerate(phon_label):
                # print('******** SYL ********')
                # if phoneme is nonvowel, skip to next phoneme
                if not vowels[all_phon_counter]:
                    # print('!!! nonvowel')
                    continue
                    
                # build the filename of the phoneme
                filename_phon = ''.join([filename_base, '.s', format(all_phon_counter, '02d'), '_', phon[0], 
                                         str(all_phon_counter), csv_input_file_extension])
                # print('filename_phon ', filename_phon)
                
                # check if phon file is in test_log.txt
                # phon file is not empty, so:
                if filename_phon in test_log:
                    # get duration of original phoneme contour
                    phon_dur_orig = phon_time[all_phon_counter, 1] - phon_time[all_phon_counter, 0]
                    # print('phon_dur_orig = ', phon_dur_orig)
                    
                    # get duration of new phoneme contour
                    # get number of lines from file
                    fphon = open(os.path.join(phon_input_directory_path, filename_phon), 'r')
                    phon_vals = fphon.read().strip().split('\n')
                    # print('phon_vals=', phon_vals)
                    num_lines = len(phon_vals)
                    # print('num_lines= ', num_lines)
                    
                    # multiply by window step size to get time, and add half a window
                    phon_dur_new = (num_lines * step_s) + step_s
                    # print('phon_dur_new = ', phon_dur_new)
                    
                    # calculate the diff (will be positive if new duration is larger)
                    diff = phon_dur_new - phon_dur_orig
                    # print('diff = ', diff)
                    # print('********')
                    
                    # # append phon_vals to end of list, to later calculate the minimum f0 in the vowels regions
                    # # print(phon_vals)
                    # phon_vals_all += phon_vals
                    # # print('phon_vals_all ', phon_vals_all)
                    
                # phon file is either missing or empty, so make diff = 0
                else:
                    diff = 0
                    # print('!!! file missing: ', filename_phon)
                    # print('diff = ', diff)
                
                # print('new_phon_time BEFORE UPDATE', new_phon_time)
                # run through the new_phon_time list, adding the (positive/negative) diff to end time of phon, 
                # and start and end times of all subsequent phonemes
                # new_phon_time = [time if i-1 < all_phon_counter * 2 else time + diff for i, time in enumerate(
                #     new_phon_time)]
                new_phon_time = [time if i-1 < all_phon_counter else time + diff for i, time in enumerate(
                    new_phon_time)]
                # print('new_phon_time AFTER UPDATE = ', new_phon_time)
                
            
            # convert phon_time to a list
            # phon_time = [val for i, val in enumerate(phon_time)]  
            # phon_time = list(phon_time.flatten(order='C')) # don't need both columns, it just duplicates the times
            phon_time = [0.0] + [x for x in phon_time[:, 1]]  # just get second column, plus add a 0.0 at the start
            # print(phon_time)
            # append phon_time, new_phon_time to outfiles            
            [f_phon_time.write(str(val) + ',') if i < len(phon_time) - 1 else f_phon_time.write(str(val)) for i, 
                                                                                                          val in enumerate(phon_time)]
            f_phon_time.write('\n')
            
            
            
            # [f_new_phon_time.write(str(val) + ',') for val in new_phon_time]
            [f_new_phon_time.write(str(val) + ',') if i < len(new_phon_time) - 1 else f_new_phon_time.write(str(val)) for i, 
                                                                                                          val in enumerate(new_phon_time)]
            f_new_phon_time.write('\n')
            # print to stdout to check
            # [sys.stdout.write(str(val) + ',') for val in new_phon_time]
            # sys.stdout.write('\n')
            
            # print wav file name to file
            f_wav_files.write(wav_filename + '\n')
            
            # # convert phon_vals_all to numpy array of integers, get minimum and write to text file
            # # print(phon_vals_all)
            # phon_vals_all = [int(x) for x in phon_vals_all]
            # phon_vals_all = np.array(phon_vals_all)
            # # print(phon_vals_all)
            # phon_vals_min = phon_vals_all.min()
            # # print(phon_vals_min) 
        
            # build original sdif file name
            sdif_glob_filepath = os.path.join(sdif_input_directory_path, filename_base + '.*')
            # print('sdif_glob_filepath ', sdif_glob_filepath)
            sdif_filepath_list = glob.glob(sdif_glob_filepath)
            # print('sdif_filepath_list ', sdif_filepath_list)
            sdif_filepath = sdif_filepath_list[0] 
            # print('sdif_filepath ', sdif_filepath)
            
            # read in time/f0/harm from corresponding sdif file - returns tuple of ndarrays
            (f0time, f0harm, f0val) = get_f0_info(sdif_filepath)
            # filter lines when harm >= threshold e.g. 0.3
            f0val = f0val[(f0harm >= harmonicity_threshold).nonzero()]            
            # get minimum f0 in remaining set and write to file
            f0val_min = f0val.min()
            # print(f0val_min)
            
            # if the value is 50 or over then write to file, otherwise default to 50 (eliminates erroneous low values)
            if f0val_min >= 150:
                f_phon_vals_min.write(str(f0val_min) + '\n')
            else:
                f_phon_vals_min.write(str(50) + '\n')            
                

# close the files         
f_test_log.close()
f_wav_files.close()
f_phon_time.close()
f_new_phon_time.close()
f_phon_vals_min.close()

print('done')

done


<h3>Read in syllable timecode csv files and perform timestretch on WAV files

In [240]:
'''
# def alignSnds(sndfile_to_be_aligned, outfile, temporal_positions,
#               winsize,  svpcmd=svp_executable, shape=False,
#               transients=False, oversamp=8, vudem=3, transdet_minren=None, verbose=False) :
#     use supervp to align sndfile_to_be_aligned to sndfile by means of stretching
#     the first to align to the second.

# time stretch whole source soundfile
# param: sndfile_to_be_aligned: source soundfile from olivia2006 dataset
# param: outfile: output file
# param: temporal_positions: list of tuples of time before, time after

shares vars with above code, so run all
'''

# bug: before times has duplicates in it.. 'phon_time.txt'

# open files and split into list of lines
f_wav_files = open(os.path.join(output_directory, filename_wav_files), 'r')
wav_files = f_wav_files.read().split('\n')
f_phon_time = open(os.path.join(output_directory, filename_phon_time), 'r')
phon_times = f_phon_time.read().split('\n')
f_new_phon_time = open(os.path.join(output_directory, filename_new_phon_time), 'r')
new_phon_times = f_new_phon_time.read().split('\n')
f_phon_vals_min = open(os.path.join(output_directory, filename_phon_vals_min), 'r')
phon_vals_min = f_phon_vals_min.read().split('\n')

# debug
# wav_files = ['Olivia2006.e01.p01.i00.1.wav']
# print(wav_files)

# for each file in enumerate(wav_files)
for i, wav_file in enumerate(wav_files):
    # skip if wav_file is empty:
    if not wav_file:
        continue
#     build the full path for sndfile_to_be_aligned
    wav_file_path = os.path.join(wav_input_directory_path, wav_file)
    # print('wav_file_path ', wav_file_path)
#     get before times and after times from corresponding row of the two phon_time files, and put in lists
    before_times = phon_times[i].split(',')
    before_times = [float(x) for x in before_times if x]
    # print('before_times ', before_times)
    end_times = new_phon_times[i].split(',')
    end_times = [float(x) for x in end_times if x]
    # print('end_times ', end_times)
#     make temporal_positions list of tuples from corresponding items from each list
#     temporal_positions = list(zip(before_times, end_times))

    # loop through zipped list backwards, removing any tuples that have identical elements
    # this is to solve RuntimeError("do not support alignment with zero size segments.")
    # length = len(temporal_positions)
    # # print(length)
    # for j in range(length - 1, -1, -1):
    #     print(j)
    #     print(temporal_positions[j])
    #     print(temporal_positions[j][0])
    #     print(temporal_positions[j][1])
    #     if temporal_positions[j][0] == temporal_positions[j][1]:
    #         del temporal_positions[j]
    #         print('del ', j)

    # [del temporal_positions[i] if temporal_positions[i][0] == temporal_positions[i][1] for i, x in enumerate(temporal_positions)]
    # print(temporal_positions)

    # convert cleaned list of tuples to list of two-element arrays
    temporal_positions = [np.array(x) for x in zip(before_times, end_times)]
    # print('temporal_positions ', temporal_positions)
    # print('len ', len(temporal_positions))

#     make outfile name from infile name
#     print('wav_file ', wav_file)
    wav_file_out, ext = os.path.splitext(wav_file)
    wav_file_out = wav_file_out + '.OUT' + ext
    # print('wav_file_out ', wav_file_out)
    wav_file_out_path = os.path.join(output_directory, wav_file_out)
#     make winsize
    winsize = 4 * (1/float(phon_vals_min[i]))
    # winsize = 800 # works much better
    # print('i ', i)
    # print('winsize ', winsize)
#     call alignSnds with above params: shape=True, winsize = 4 x f0_min of unvoiced regions of THIS (source) wav file
    alignSnds(wav_file_path, wav_file_out_path, temporal_positions, winsize, shape=True, verbose=True)
    # print('******************')

print('done')


08/29/2018 20:31:19 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0231279s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e01.p01.i00.1.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p01.i00.1.OUT.wav


08/29/2018 20:31:20 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0232535s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e02.p01.i00.62.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e02.p01.i00.62.OUT.wav


08/29/2018 20:31:21 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0225101s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e03.p01.i00.122.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e03.p01.i00.122.OUT.wav


08/29/2018 20:31:21 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0207914s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e04.p01.i00.182.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e04.p01.i00.182.OUT.wav


08/29/2018 20:31:22 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0217512s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e05.p01.i00.242.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e05.p01.i00.242.OUT.wav


...

08/29/2018 20:31:34 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0235492s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e05.p10.i00.298.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e05.p10.i00.298.OUT.wav


08/29/2018 20:31:35 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0242392s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e06.p10.i00.360.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e06.p10.i00.360.OUT.wav


08/29/2018 20:31:35 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.08s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e07.p10.i00.421.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e07.p10.i00.421.OUT.wav


08/29/2018 20:31:36 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD35281.par -M0.0257734s -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e08.p10.i00.481.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p10.i00.481.OUT.wav


done


<h3>Read in time stretched WAV files, extract f0 of each, and output result SDIF files

In [241]:
'''
def calc_f0_swipe(infile, outfile, f0min=50, f0max=450, time_step=0.005,
                  harmonicity_threshold=None,
                  use_spline_interp=False, ana_freq_limit=12500, music_mode=False,
                   remove_low_harm_f0=True, channel=0, verbose=False):

    use swipe with given parameters to create the f0 analysis of infile and store the result in an sdif file

    raises RuntimeError in case of errors

'''

# input directory is former output_directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO'
input_directory_path = os.path.join(wav_input_directory_path, 'timestretch')
# input_directory_path = wav_input_directory_path  #debug
input_directory = os.fsencode(input_directory_path)

# define new output directory and files
output_directory_path = os.path.join(wav_input_directory_path, 'timestretch_f0extract')
# output_directory_path = os.path.join(wav_input_directory_path, 'original_f0extract')  #debug
if not os.path.exists(output_directory_path):
    os.mkdir(output_directory_path)

# for each wav file in directory
for file in os.listdir(input_directory):
    # get filename
    filename = os.fsdecode(file)
    # get filepath and decode from bytes to string
    # filepath = os.path.abspath(file).decode("utf-8") 
    filepath = os.path.join(input_directory_path, filename)
    print(filepath)
    
    if filename.endswith('.wav'):
        # build outfile sdif filename
        sdif_file_out, ext = os.path.splitext(filename)
        sdif_file_out = sdif_file_out + '.sdif'
        sdif_file_out_path = os.path.join(output_directory_path, sdif_file_out)
        
        # extract f0 and write to sdif
        calc_f0_swipe(filepath, sdif_file_out_path)


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/.DS_Store
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p01.i00.1.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p03.i00.14.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p05.i00.26.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p10.i00.56.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e02.p01.i00.62.OUT.wav


...

/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p01.i00.427.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p03.i00.439.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p05.i00.451.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p10.i00.481.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/phon_time.txt
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/phon_time_new.txt
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/phon_vals_min.txt
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/wav_files.txt


<h3>Read in timestretched wav, timestretched f0 sdif, and target f0 array, and output transposed f0 WAV file

In [245]:
'''
i need to:

piece together all f0 phoneme files into one continuous sequence, including unvoiced sections (get durations from 
phon_time), and put into an array


call f0_transpose to convert, passing in timestretched wav, timestretched f0 sdif, and target f0 array
as well as outfile wav file name, and winsize scalar

'''


# *************
# START CONFIG

# for each 
# get list 
# path to CSV input files directory
csv_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006/f0_raw_phoneme'
# Olivia2006.e01.p01.i00.s01_s e.csv
csv_input_file_extension = '.csv'

# path to predicted F0 CSV input files directory
phon_input_directory_path = '/Users/robinson/Downloads/data/pred/' + dirr + '/phonemes'
# Olivia2006.e01.p01.i00.s01_s e1.csv
phon_input_file_extension = '.csv'

# path to MAT input files directory
mat_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006'
# Olivia2006.e01.p01.i00.mat
mat_input_file_extension = '.mat'

# path to WAV input files directory
raw_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/'
# Olivia2006.e01.p01.i00.1.wav
raw_input_file_extension = '.wav'

# path to timestretched WAV input files directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch'
# Olivia2006.e01.p01.i00.1.wav
wav_input_file_extension = '.wav'

# path to SDIF f0 input files directory
sdif_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch_f0extract'
# Olivia2006.e01.p01.i00.mat
sdif_input_file_extension = '.sdif'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# define output directory and files
output_directory = os.path.join(raw_input_directory_path, 'f0transpose')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# delete all files in f0transpose and compare
for folder in [output_directory, os.path.join(raw_input_directory_path, 'compare')]:
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

# open test_log.txt and split into list of lines
f_test_log = open(os.path.join(csv_input_directory_path, 'out', 'test_log.txt'), 'r')
test_log = f_test_log.read().split('\n')

filename_phon_vals_min = 'phon_vals_min.txt'
f_phon_vals_min = open(os.path.join(wav_input_directory_path, filename_phon_vals_min), 'r')
phon_vals_min = f_phon_vals_min.read().split('\n')

# define phrase range
phrase_from = 1
phrase_to = 10  #10
phrase_list = [1, 3, 5, 10]
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8  #8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# set sample rate 
step_s = 0.005 # 5ms, which I assume is what was used to sample the file

# END CONFIG
# *************

def zero_helper(y):
    return y == 0, lambda z: z.nonzero()[0]

# file counter 
i = 0

# for each wavfile that we want to treat..
# for each phrase
# for p in range(phrase_from, phrase_to + 1):
for p in phrase_list: 
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            
            # open the mat file 
            mat_filename = ''.join([filename_base, mat_input_file_extension])
            mat_filepath = os.path.join(mat_input_directory_path, mat_filename)
            mat_dict = loadmat(mat_filepath)
            
            syll_label = mat_dict['syll_label']
            syll_label = syll_label.reshape((syll_label.shape[1],))        
            # print(syll_label.shape)
            # print(syll_label)

            # I want syll_time.shape (11, 2) BUT with the start and end times in different 'columns' - just transpose!
            syll_time = mat_dict['syll_time']
            # print('syll_time.shape', syll_time.shape)
            syll_time = syll_time.T
            # print('syll_time.shape', syll_time.shape)
            # print(syll_time)            
            
            # get list of phonemes
            phon_label = mat_dict['phone_label']
            phon_label = phon_label.reshape((phon_label.shape[1],))
            # print('phon_label ', phon_label)
            
            # get phon_time from dict and transpose to column
            phon_time = mat_dict['phone_time'].T
            # print('phon_time ', phon_time)
            
            # make list of true/false to indicate vowels(true)/nonvowel(false)
            vowel_phonemes = ['e~', '9~', 'a~', 'o~', 'i', 'e', 'E', 'a', 'A', 'O', 'o', 'u', 'y', '2', '9', '@']
            # [a if C else b for i in items]
            vowels = [True if phon[0] in vowel_phonemes else False for i, phon in enumerate(phon_label)]
            # print('vowels ', vowels)
            
            # create list to hold sequence of vowels and unvoiced f0 contour values
            all_contours = []
            
            # for each syllable in the wav file
            for all_syll_counter, syll in enumerate(syll_label):
                
                # get syll_time start and end times
                # print('i = ', i)
                syll_start_time = syll_time[all_syll_counter, 0]
                syll_end_time = syll_time[all_syll_counter, 1]
                
                phone_id_list = []
                # for each phone label in phon_label (this mat file)
                for j, label in enumerate(phon_label):
                    # get start/end times from phone_time
                    phone_start_time = phon_time[j, 0]
                    phone_end_time = phon_time[j, 1]
                    # if start time => syll_start_time AND end time <= syll_end_time then this phoneme is in the syllable
                    if phone_start_time >= syll_start_time and phone_end_time <= syll_end_time:
                        # add phoneme label id to a list (to use to reference label and start/end times)
                        phone_id_list.append(j)
                        
                # for each phoneme in the syllable file
                for all_phon_counter in phone_id_list:
                # for all_phon_counter, phon in enumerate(phon_label):
                    # print('******** SYL ********')
                    # build the filename of the phoneme (which may not exist for this syllable)
                    filename_phon = ''.join([filename_base, '.s', format(all_syll_counter, '02d'), '_',
                                             phon_label[all_phon_counter][0], str(all_phon_counter), 
                                             csv_input_file_extension])
                    print('filename_phon', filename_phon)

                    # if phoneme is not a vowel, or if phon file is a vowel but there's no file for it
                    # get the length of the unvoiced phoneme and add zeroes to all_contours list                
                    if not vowels[all_phon_counter] or not os.path.isfile(os.path.join(phon_input_directory_path, 
                                                                                       filename_phon)):
                        # print('!!! nonvowel or file DOES NOT exist')
                        time = phon_time[all_phon_counter, 1] - phon_time[all_phon_counter, 0]
                        # print('time ', time)
                        num_zeroes = int(time // step_s)
                        # print('num_zeroes ', num_zeroes)
                        all_contours += [0 for _ in range(num_zeroes)]
                    # if phoneme is vowels, add its contents to the all_contours list
                    else:
                        # print('!!! vowel and file exists')
                        # build the source file path
                        source_file_path = os.path.join(phon_input_directory_path, filename_phon)
                        # print('source_file_path',source_file_path)

                        # load the source file and extract vars
                        source_f0_raw = np.loadtxt(source_file_path, dtype='int')
                        # print('shape ', source_f0_raw.shape)

                        # iterate over the numpy array, adding items to the all_contours list
                        all_contours += [source_f0_raw[x] for x in range(source_f0_raw.shape[0])]

                        
            # print('all_contours BEFORE INTERP', all_contours)
            if interp:
                # convert strings of zeroes to linearly interpolated values between neighbouring non-zero vals
                y = np.array(all_contours)
                # print('y',y)
                zeroes, lambda_func = zero_helper(y)
                y[zeroes] = np.interp(lambda_func(zeroes), lambda_func(~zeroes), y[~zeroes])
                all_contours = y.tolist()
            # print('all_contours AFTER INTERP', all_contours)
                
            # convert into a single array to pass to f0_transpose method - not sure if needed..
            all_times = [x * step_s for x in range(len(all_contours))]
            all_times_contours = list(zip(all_times, all_contours))
            
            # print(all_times_contours)
            
            # print('*** length = ', len(all_times_contours))
            for j in range(len(all_times_contours) - 1, 0, -1):
                # print(i)
                if all_times_contours[j][1] == 0:
                    del all_times_contours[j]
    
            all_times_contours = np.array(all_times_contours)
            # .reshape(-1, 2)
            # print(all_times_contours)
            
            # build timestretched input wav file name and path - use glob to handle number           
            wav_glob_filepath = os.path.join(wav_input_directory_path, filename_base + '.*')
            # print('wav_glob_filepath ', wav_glob_filepath)
            wav_filepath_list = glob.glob(wav_glob_filepath)
            # print('wav_filepath_list ', wav_filepath_list)
            wav_filepath = wav_filepath_list[0] 
            # print('wav_filepath ', wav_filepath)
            _, wav_filename = os.path.split(wav_filepath)
            wav_filename_noext, _ = os.path.splitext(wav_filename)
            
            
            # build outfile wav file name and path
            wav_filename_out = ''.join([wav_filename_noext, emotion_code, wav_input_file_extension])
            wav_filename_out_path = os.path.join(output_directory, wav_filename_out)
            print('writing... ', wav_filename_out)
            
            # build sdif file name
            sdif_glob_filepath = os.path.join(sdif_input_directory_path, filename_base + '.*')
            # print('sdif_glob_filepath ', sdif_glob_filepath)
            sdif_filepath_list = glob.glob(sdif_glob_filepath)
            # print('sdif_filepath_list ', sdif_filepath_list)
            sdif_filepath = sdif_filepath_list[0] 
            # print('sdif_filepath ', sdif_filepath)
            
            # set winsize scalar
            # winsize = 800  # worked before, but dunno why
            winsize = 4 * (1/float(phon_vals_min[i]))
            # winsize = 2 * (1/float(phon_vals_min[i]))
            # winsize = 4 * (1/float(50))
            
            # call transpose_f0
            transpose_f0(wav_filepath, sdif_filepath, all_times_contours, wav_filename_out_path, winsize,  
                         harmonicity_threshold=harmonicity_threshold, remove_low_harm_f0=True)
            
            # increment counter
            i += 1
            
print('done')

filename_phon Olivia2006.e01.p01.i00.s00_###0.csv
filename_phon Olivia2006.e01.p01.i00.s01_s1.csv
filename_phon Olivia2006.e01.p01.i00.s01_e2.csv
filename_phon Olivia2006.e01.p01.i00.s02_9~3.csv
filename_phon Olivia2006.e01.p01.i00.s03_s4.csv
filename_phon Olivia2006.e01.p01.i00.s03_O5.csv
filename_phon Olivia2006.e01.p01.i00.s03_l6.csv
filename_phon Olivia2006.e01.p01.i00.s04_d7.csv
filename_phon Olivia2006.e01.p01.i00.s04_a8.csv
filename_phon Olivia2006.e01.p01.i00.s05_#9.csv
filename_phon Olivia2006.e01.p01.i00.s06_a10.csv
filename_phon Olivia2006.e01.p01.i00.s07_S11.csv
filename_phon Olivia2006.e01.p01.i00.s07_212.csv
filename_phon Olivia2006.e01.p01.i00.s08_v13.csv
filename_phon Olivia2006.e01.p01.i00.s08_214.csv
filename_phon Olivia2006.e01.p01.i00.s09_g15.csv
filename_phon Olivia2006.e01.p01.i00.s09_R16.csv
filename_phon Olivia2006.e01.p01.i00.s09_i17.csv
filename_phon Olivia2006.e01.p01.i00.s10_###18.csv


ValueError: array of sample points is empty

In [243]:
# copy original source files to directory to compare

# path to raw WAV files directory
raw_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/'
# Olivia2006.e01.p01.i00.1.wav

# path to timestretched WAV files directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch'

# path to timestretched f0 tranformed files directory
joy_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/f0transpose'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# define output directory and files
output_directory = os.path.join(raw_input_directory_path, 'compare')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# define phrase range
phrase_from = 1
phrase_to = 10  #10
phrase_list = [1, 3, 5, 10]
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8  #8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# for each wavfile that we want to treat..
# for each phrase
# for p in range(phrase_from, phrase_to + 1):
for p in phrase_list: 
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            
            # build input wav file name and path - use glob to handle number           
            wav_glob_filepath = os.path.join(raw_input_directory_path, filename_base + '.*')
            # print('wav_glob_filepath ', wav_glob_filepath)
            wav_filepath_list = glob.glob(wav_glob_filepath)
            # print('wav_filepath_list ', wav_filepath_list)
            wav_filepath = wav_filepath_list[0] 
            # print('wav_filepath ', wav_filepath)
            _, wav_filename = os.path.split(wav_filepath)
            
            # copy the raw file to the output directory
            copyfile(wav_filepath, os.path.join(output_directory, wav_filename))
            
            
# copy all timestretched wav files to compare directory
wav_input_directory = os.fsencode(wav_input_directory_path)
# for each wav file in directory
for file in os.listdir(wav_input_directory):
    # get filename
    filename = os.fsdecode(file)
    if filename.endswith('.wav'):
        # get filepath and decode from bytes to string
        # filepath = os.path.abspath(file).decode("utf-8") 
        filepath = os.path.join(wav_input_directory_path, filename)
        
        # copy the timestretched wav file to the output directory
        #copyfile(filepath, os.path.join(output_directory, filename))
    
    
# copy all f0 transformed wav files to compare directory
joy_input_directory = os.fsencode(joy_input_directory_path)
# for each wav file in directory
for file in os.listdir(joy_input_directory):
    # get filename
    filename = os.fsdecode(file)
    if filename.endswith('.wav'):
        # get filepath and decode from bytes to string
        # filepath = os.path.abspath(file).decode("utf-8") 
        filepath = os.path.join(joy_input_directory_path, filename)
        
        # copy the timestretched wav file to the output directory
        copyfile(filepath, os.path.join(output_directory, filename))

print('done')

done


In [244]:
# import numpy as np
# 
# def zero_helper(y):
#     return y == 0, lambda z: z.nonzero()[0]
# 
# y[zeroes] = np.interp(x(zeroes), x(~zeroes), y[~zeroes])
# 
# print(y.round(2))